In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("../")

In [4]:
from data_processing.start_experiment import *
from data_processing.read_data import *
from data_processing.gaussian import *

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign
from emukit.core.initial_designs.latin_design import LatinDesign
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
import GPy
from emukit.sensitivity.monte_carlo import MonteCarloSensitivity
import time

## Create GP to Model Single Cell Rate

Load the Latin data where only gamma and gradnoise are varied

In [5]:
num_experiments = 100
parameter_list = ['gamma', 'gradnoise']
parameter_space = ParameterSpace([DiscreteParameter('gamma',list(range(0,25))),
                                 ContinuousParameter('gradnoise',0.25,0.99)])
file_names = ["gradient_latin/latin_{}.par".format(i) for i in range(num_experiments)]
data_files = ["gradient_latin/data_cellcount_{}.txt".format(i) for i in range(num_experiments)]

X = get_parameter_array(file_names,parameter_list)

# Some files have no data; don't get those
unrun_files = get_no_data(data_files)
X_no_data = X[unrun_files,:]

X = np.delete(X,unrun_files,axis=0)

In [6]:
Y = get_rewards(data_files,function_at_last_time(single_cell_rate))

KeyboardInterrupt: 

In [ ]:
gpy_model = GPy.models.GPRegression(X, Y, GPy.kern.RBF(X.shape[1]), noise_var=1)
emukit_model = GPyModelWrapper(gpy_model)


## Plot Gaussian Process Model

In [ ]:
plot_gaussian_process(np.linspace(0,25, 24),0,parameter_space,emukit_model)

In [ ]:
plot_gaussian_process(np.linspace(0.25, 0.99, 74),1,parameter_space,emukit_model)

In [ ]:
print(np.array(sorted(np.hstack((X,Y)), key = lambda x : x[2])))


The results for gradnoise seem very noisy and not predictive of multicellularity. 
We will try again only with data points with gamma between 0 and 5 (such that its effect is minimized). 

In [ ]:
filtered_data = np.hstack((X,Y))
filtered_data = filtered_data[[x[0] < 6 for x in filtered_data]]
X_f = filtered_data[:, :2]
Y_f = filtered_data[:, 2:]

gpy_model = GPy.models.GPRegression(X_f, Y_f, GPy.kern.RBF(X_f.shape[1]), noise_var=1)
emukit_model = GPyModelWrapper(gpy_model)
plot_gaussian_process(np.linspace(0.25, 0.99, 74),1,parameter_space,emukit_model)


Contrary to what the original paper asserts the gradient noise seems to have little predictive value (do formal sensitivity analysis?). However, this might be due to the fixed execution of one season. (Investigate?)

Based on what we learned from this coexistince might be possible for gamma values around 5. We will now try to fit a GP to season duration trying to optimize for co-existence. 

In [9]:
num_experiments = 10
parameter_list = ['season_duration']
constant_parameters = {
    "mcs" : 50000,
    "cell_size" : 50,
    "gamma" : 5,
    "T" : 16,
    "storage_stride":10000
}
parameter_space = ParameterSpace([DiscreteParameter('season_duration',list(range(1000, 10001, 25)))])


In [10]:
run_simulation_latin(parameter_space, num_experiments, 0, constant_parameters)


array([[5950],
       [9550],
       [6850],
       [2350],
       [3250],
       [4150],
       [8650],
       [7750],
       [1450],
       [5050]])

Read in the files

In [8]:
file_names = ["latin_{}.par".format(i) for i in range(num_experiments)]
data_files = ["data_cellcount_{}.txt".format(i) for i in range(num_experiments)]

X = get_parameter_array(file_names,parameter_list)

# Some files have no data; don't get those
unrun_files = get_no_data(data_files)
X_no_data = X[unrun_files,:]

X = np.delete(X,unrun_files,axis=0)

FileNotFoundError: [Errno 2] No such file or directory: '../data/parameters/data_cellcount_0.par'

In [ ]:
Y = get_rewards(data_files,average_over_time(single_cell_rate_entropy))

Create initial GP Model

In [11]:
gpy_model = GPy.models.GPRegression(X, Y, GPy.kern.RBF(X.shape[1]), noise_var=1)
emukit_model = GPyModelWrapper(gpy_model)
plot_gaussian_process(np.array(range(1000, 10001, 25)),0,parameter_space,emukit_model)

NameError: name 'X_f' is not defined

In [ ]:
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

acquisition = ExpectedImprovement(parameter_space)
bayesopt_loop = BayesianOptimizationLoop(model = model_emukit,
                                         space = parameter_space,
                                         acquisition = acquisition,
                                         batch_size = 1)

max_iterations = 30
bayesopt_loop.run_loop(f, max_iterations)


## Perform Sensitivity Analysis